In [1]:
%pip install scipy
%pip install plotly
%pip install pandas

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
from scipy.cluster.hierarchy import linkage, leaves_list


def create_heatmap_without_sample_clustering(df_heatmap, color='RdBu_r'):
    """根据基因表达矩阵创建热图

    Args:
        df_heatmap (pd.DataFrame): 基因表达矩阵
        color (str, optional): 颜色. Defaults to 'RdBu_r'.
    """

    # 第一列改名为gene_id，并设置为索引
    df_heatmap = df_heatmap.rename(columns={df_heatmap.columns[0]: 'gene_id'})
    df_heatmap = df_heatmap.set_index('gene_id')

    # 数据Z-score标准化,axis=0表示按列标准化,axis=1表示按行标准化
    # df_heatmap_normalized = df_heatmap.apply(lambda x: (x - x.mean()) / x.std(), axis=1)
    # df_heatmap_normalized = df_heatmap_normalized.apply(lambda x: np.clip(x, -3, 3), axis=1)

    # log2转换
    df_heatmap_normalized = df_heatmap.apply(lambda x: np.log2(x + 1))

    # 对基因（行）进行层次聚类
    gene_linkage = linkage(df_heatmap_normalized, method='average')
    gene_order = leaves_list(gene_linkage)

    # # 对样本（列）进行层次聚类
    # sample_linkage = linkage(df_heatmap_normalized.T, method='average')
    # sample_order = leaves_list(sample_linkage)

    # 重排数据矩阵
    df_heatmap_clustered = df_heatmap_normalized.iloc[gene_order]
    # df_heatmap_clustered = df_heatmap_normalized.iloc[gene_order,sample_order]

    # 创建热图
    fig = go.Figure(data=go.Heatmap(
        z=df_heatmap_clustered,
        x=df_heatmap_clustered.columns,
        y=df_heatmap_clustered.index,
        colorscale=color,
        # zmin=-3,  # 设置颜色映射的最小值
        # zmax=3,   # 设置颜色映射的最大值
        hovertemplate='Sample: %{x}<br>Gene: %{y}<br>Exp: %{z:.3f}<extra></extra>'
    ))

    fig.update_layout(
        xaxis_title="Sample",
        yaxis_title="Gene",
        width=1200,
        height=900,
        xaxis=dict(
            tickfont=dict(size=8),  # 设置字体大小为8
            tickangle=45  # 将刻度标签旋转45度
        )
)

    fig.write_html("heatmap网页效果参考.html")

    return fig


# 加载数据
df_heatmap = pd.read_csv('./Gene_Expression_Matrix.tsv',sep='\t').copy()

# 测试
create_heatmap_without_sample_clustering(df_heatmap)